# ネットワークモデルの実装

In [1]:
#パッケージのインポート
import glob
import os.path as osp
import random
import numpy as np
import json
from PIL import Image
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import models,transforms
import cv2 
import torch.nn.init as init

## モジュールvggを実装

In [11]:
#35層にわたる、vggモジュールを作成
from torch import conv2d


def make_vgg():
    layers=[]
    in_channels=3 #色チャネル数

    #vggモジュールで使用する畳み込み層やマックスプーリングのチャネル数
    cfg=[64,64,"M",128,128,"M",256,256,256,"MC",
          512,512,512,"M",512,512,512]

    for v in cfg:
        if v=="M":
            layers+=[nn.MaxPool2d(kernel_size=2,stride=2)]
        elif v=="MC":
            #ceilは出力サイズを計算結果(float)に対して、切り上げで整数にするモード
            #デフォルトでは出力サイズを計算結果(float)に対して、切り下げで整数にする
            #floorモード
            layers+=[nn.MaxPool2d(kernel_size=2,stride=2,ceil_mode=True)]
        else:
            conv2d=nn.Conv2d(in_channels,v,kernel_size=3,padding=1)
            layers+=[conv2d, nn.ReLU(inplace=True)]
            in_channels=v

    pool5=nn.MaxPool2d(kernel_size=3,stride=1,padding=1)
    conv6=nn.Conv2d(512,1024,kernel_size=3,padding=6,dilation=6)
    conv7=nn.Conv2d(1024,1024,kernel_size=1)
    layers+=[pool5,conv6,nn.ReLU(inplace=True),conv7,nn.ReLU(inplace=True)]

    return nn.ModuleList(layers)



In [5]:
#動作確認
vgg_test=make_vgg()
print(vgg_test)

ModuleList(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
  (17): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1

## モジュールextraを実装


In [14]:
def make_extras():
    layers=[]
    in_channels=1024 #vggモジュールから出力された、extraに入力さされる画像チャネル数

    #extraモジュールの畳み込み層のy種留守うを設定するコンフィギュレーション
    cfg=[256,512,128,256,128,256,128,256]

    layers+=[nn.Conv2d(in_channels,cfg[0],kernel_size=1)]
    layers+=[nn.Conv2d(cfg[0],cfg[1],kernel_size=3,stride=2,padding=1)]
    layers+=[nn.Conv2d(cfg[1],cfg[2],kernel_size=1)]
    layers+=[nn.Conv2d(cfg[2],cfg[3],kernel_size=3,stride=2,padding=1)]
    layers+=[nn.Conv2d(cfg[3],cfg[4],kernel_size=1)]
    layers+=[nn.Conv2d(cfg[4],cfg[5],kernel_size=3)]
    layers+=[nn.Conv2d(cfg[5],cfg[6],kernel_size=1)]
    layers+=[nn.Conv2d(cfg[6],cfg[7],kernel_size=3)]

    return nn.ModuleList(layers)

In [12]:
#動作確認
extras_test=make_extras()
print(extras_test)

ModuleList(
  (0): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
  (1): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (2): Conv2d(512, 128, kernel_size=(1, 1), stride=(1, 1))
  (3): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (4): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1))
  (5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
  (6): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1))
  (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
)


## モジュールlocとモジュールconfを実装

In [32]:
#デフォルトボックスのオフセットを出力するloc_layers,
#デフォルトボックスに対する各クラスの信頼度confidenceを出力するconf_layersを作成

def make_loc_conf(num_classes=21,bbox_aspect_num=[4,6,6,6,4,4]):
    loc_layer=[]
    conf_layer=[]

    #VGGの22層目,conv4_3(source1)に対する畳み込み層
    loc_layer+=[nn.Conv2d(512,bbox_aspect_num[0]*4, kernel_size=3, padding=1)]
    conf_layer+=[nn.Conv2d(512,bbox_aspect_num[0]*num_classes, kernel_size=3,padding=1)]
    
    #VGGの最終層(source2)に対する畳み込み層
    loc_layer+=[nn.Conv2d(1024,bbox_aspect_num[1]*4, kernel_size=3, padding=1)]
    conf_layer+=[nn.Conv2d(1024,bbox_aspect_num[1]*num_classes, kernel_size=3, padding=1)]
    
    #extraの(source3)に対する畳み込み層
    loc_layer+=[nn.Conv2d(512, bbox_aspect_num[2]*4,kernel_size=3,padding=1)]
    conf_layer+=[nn.Conv2d(512,bbox_aspect_num[2]*num_classes, kernel_size=3,padding=1)]

    #extraの(source4)に対する畳み込み層
    loc_layer+=[nn.Conv2d(256, bbox_aspect_num[3]*4,kernel_size=3,padding=1)]
    conf_layer+=[nn.Conv2d(256,bbox_aspect_num[3]*num_classes,kernel_size=3,padding=1)]

    #extraの(source5)に対する畳み込み層
    loc_layer+=[nn.Conv2d(256, bbox_aspect_num[4]*4,kernel_size=3,padding=1)]
    conf_layer+=[nn.Conv2d(256,bbox_aspect_num[4]*num_classes,kernel_size=3,padding=1)]

    #extraの(source6)に対する畳み込み層
    loc_layer+=[nn.Conv2d(256, bbox_aspect_num[5]*4,kernel_size=3,padding=1)]
    conf_layer+=[nn.Conv2d(256,bbox_aspect_num[5]*num_classes,kernel_size=3,padding=1)]

    
    return nn.ModuleList(loc_layer),nn.ModuleList(conf_layer)

    
    

In [20]:
#動作確認
loc_test,conf_test=make_loc_conf()
print(loc_test)
print(conf_test)

ModuleList(
  (0): Conv2d(512, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): Conv2d(1024, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (2): Conv2d(512, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): Conv2d(256, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): Conv2d(256, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): Conv2d(256, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)
ModuleList(
  (0): Conv2d(512, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): Conv2d(1024, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (2): Conv2d(512, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): Conv2d(256, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): Conv2d(256, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): Conv2d(256, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)


## L2Norm層を実装

In [29]:
#convC4_3からの出力をscale=20のL2Normで正規化する層
class L2Norm(nn.Module):
    def __init__(self,input_channels=512,scale=20):
        super(L2Norm,self).__init__() #親クラスのコンストラクタを実行
        self.weight=nn.Parameter(torch.Tensor(input_channels))
        self.scale=scale #係数weightの初期値として設定する値
        self.reset_parameters() #パラメータのリセット
        self.eps=1e-10

    def reset_parameters(self):
        """ 結合パラメータの大きさscaleの値に対する初期化を実行"""
        init.constant_(self.weight, self.scale) #weightの値が全てscale(=20)になる

    def forward(self,x):
        """ 38*38の特徴量に対して、512チャンネルにわたって二乗和のルートを求めた38*38個の値を使用し、
            各特徴量を正規化してから係数を掛け算する層"""

        #各チャネルにおける38*38の特徴量のチャネル方向の二乗和を計算し、
        #さらにルートを求め、割り算して正規化する
        #normのテンソルサイズはtorch.Size([batch_num,1,38,38])になります
        norm=x.pow(2).sum(dim=1,keepdim=True).sqrt()+self.eps
        x=torch.div(x,norm)

        #係数をかける。係数はチャネルごとに一つで、512個の係数を持つ
        #self.weightのテンソルサイズはtorch.Size([512])なので
        #torch.Size([batch_num,512,38,38])まで変形します
        weights=self.weight.unsqueeze(0).unsqueeze(2).unsqueeze(3).expand_as(x)
        out=weights*x

        return out


## デフォルトボックスを実装

In [2]:
import itertools
from itertools import product
from math import sqrt


In [3]:
"""
#デフォルトボックスを出力するクラス
class DBox(object):
    def __init__(self,cfg):
        super(DBox,self).__init__()

        #初期設定
        self.image_size=cfg["input_size"] #画像サイズの300
        #[38,19,..]各sourceの特徴量マップのサイズ
        self.feature_maps=cfg["feature_maps"]
        self.num_priors=len(cfg["feature_maps"]) #sourceの個数=6
        self.steps=cfg["steps"] #[8,16,...] DBoxのピクセルサイズ
        self.min_sizes=cfg["min_sizes"] #[30,60,...]小さい正方形のDBoxのピクセルサイズ
        self.max_sizes=cfg["max_sizes"] #[60,111,...]大きい正方形のDBoxのピクセルサイズ
        self.aspect_ratios=cfg["aspect_ratios"] #長方形のDBoxのアスペクト比

    def make_dbox_list(self):
        ''' DBoxを作成する '''
        mean=[]
        #"feature_maps":[38,19,10,5,3,1]
        for k,f in enumerate(self.feature_maps):
            for i,j in product(range(f),repeat=2):

                #特徴量の画像サイズ
                #300/"steps":[8,16,32,64,100,300]
                f_k=self.image_size/self.steps[k]

                #DBoxの中心座標x,y ただし0-1で規格化している
                cx=(j+0.5)/f_k
                cy=(i+0.5)/f_k

                #アスペクト比の小さいDBox[cx,cy,width,height]
                #"min_size":[30,60,111,162,213,264]
                s_k=self.min_sizes[k]/self.image_size
                mean+=[cx,cy,s_k,s_k]

                #アスペクト比の大きいDBox[cx,cy,width,height]
                #"max_size":[60,111,162,213,264,315]
                s_k_prime=sqrt(s_k*(self.max_sizes[k]/self.image_size))
                mean += [cx,cy,s_k_prime,s_k_prime]

                #その他のアスペクト比のdefBox [cx,cy,width,height]
                for ar in self.aspect_ratios[k]:
                    mean+=[cx,cy,s_k*sqrt(ar),s_k/sqrt(ar)]
                    mean+=[cx,cy,s_k/sqrt(ar),s_k*sqrt(ar)]

            #DBoxをテンソルに変換　touch.Size([8732,4])
            output=torch.Tensor(mean).view(-1,4)

            #DBoxが画像の外にはみ出るのを防ぐため、大きさを最小0.最大1にする
            output.clamp_(max=1,min=0)

            return output
"""



In [5]:

# デフォルトボックスを出力するクラス
class DBox(object):
    def __init__(self, cfg):
        super(DBox, self).__init__()

        # 初期設定
        self.image_size = cfg['input_size']  # 画像サイズの300
        # [38, 19, …] 各sourceの特徴量マップのサイズ
        self.feature_maps = cfg['feature_maps']
        self.num_priors = len(cfg["feature_maps"])  # sourceの個数=6
        self.steps = cfg['steps']  # [8, 16, …] DBoxのピクセルサイズ
        
        self.min_sizes = cfg['min_sizes']
        # [30, 60, …] 小さい正方形のDBoxのピクセルサイズ（正確には面積）
        
        self.max_sizes = cfg['max_sizes']
        # [60, 111, …] 大きい正方形のDBoxのピクセルサイズ（正確には面積）
        
        self.aspect_ratios = cfg['aspect_ratios']  # 長方形のDBoxのアスペクト比

    def make_dbox_list(self):
        '''DBoxを作成する'''
        mean = []
        # 'feature_maps': [38, 19, 10, 5, 3, 1]
        for k, f in enumerate(self.feature_maps):
            for i, j in product(range(f), repeat=2):  # fまでの数で2ペアの組み合わせを作る　f_P_2 個
                # 特徴量の画像サイズ
                # 300 / 'steps': [8, 16, 32, 64, 100, 300],
                f_k = self.image_size / self.steps[k]

                # DBoxの中心座標 x,y　ただし、0～1で規格化している
                cx = (j + 0.5) / f_k
                cy = (i + 0.5) / f_k

                # アスペクト比1の小さいDBox [cx,cy, width, height]
                # 'min_sizes': [30, 60, 111, 162, 213, 264]
                s_k = self.min_sizes[k]/self.image_size
                mean += [cx, cy, s_k, s_k]

                # アスペクト比1の大きいDBox [cx,cy, width, height]
                # 'max_sizes': [60, 111, 162, 213, 264, 315],
                s_k_prime = sqrt(s_k * (self.max_sizes[k]/self.image_size))
                mean += [cx, cy, s_k_prime, s_k_prime]

                # その他のアスペクト比のdefBox [cx,cy, width, height]
                for ar in self.aspect_ratios[k]:
                    mean += [cx, cy, s_k*sqrt(ar), s_k/sqrt(ar)]
                    mean += [cx, cy, s_k/sqrt(ar), s_k*sqrt(ar)]

        # DBoxをテンソルに変換 torch.Size([8732, 4])
        output = torch.Tensor(mean).view(-1, 4)

        # DBoxが画像の外にはみ出るのを防ぐため、大きさを最小0、最大1にする
        output.clamp_(max=1, min=0)

        return output


In [6]:
#動作確認
#SSD300の設定
ssd_cfg={
    "num_classes":21,
    "input_size":300,
    "bbox_aspect_num":[4,6,6,6,4,4],
    "feature_maps":[38,19,10,5,3,1],
    "steps":[8,16,32,64,100,300],
    "min_sizes":[30,60,111,162,213,264],
    "max_sizes":[60,111,162,213,264,315],
    "aspect_ratios":[[2],[2,3],[2,3],[2,3],[2],[2]],
}

#DBoxの作成
dbox=DBox(ssd_cfg)
dbox_list=dbox.make_dbox_list()

#Dboxの出力を確認する
pd.DataFrame(dbox_list.numpy())

,0,1,2,3
0,0.013333,0.013333,0.100000,0.100000
1,0.013333,0.013333,0.141421,0.141421
2,0.013333,0.013333,0.141421,0.070711
3,0.013333,0.013333,0.070711,0.141421
4,0.040000,0.013333,0.100000,0.100000
5,0.040000,0.013333,0.141421,0.141421
6,0.040000,0.013333,0.141421,0.070711
7,0.040000,0.013333,0.070711,0.141421
8,0.066667,0.013333,0.100000,0.100000
9,0.066667,0.013333,0.141421,0.141421


## クラスSSDを実装する

In [33]:
#SSDクラスを作成する
class SSD(nn.Module):
    def __init__(self,phase,cfg):
        super(SSD,self).__init__()

        self.phase=phase #train or inferenceを指定
        self.num_classes=cfg["num_classes"] #クラス数=21

        #SSDのネットワークを作る
        self.vgg=make_vgg()
        self.extras=make_extras()
        self.L2Norm=L2Norm()
        self.loc,self.conf=make_loc_conf(
            cfg["num_classes"],cfg["bbox_aspect_num"])

        #DBox作成
        dbox=DBox(cfg)
        self.dbox_list=dbox.make_dbox_list()

        #推論時はクラス[Detect]を用意します
        if phase=="inference":
            self.detect=Detect()

In [34]:
#動作確認
ssd_test=SSD(phase="train",cfg=ssd_cfg)
print(ssd_test)

SSD(
  (vgg): ModuleList(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
    (17): Conv2d(256, 